In [ ]:
"""Тестирование проводим за период с 18.12.2012 по 29.04.2022, 
т.к. с сайта Мосбиржи доступна подробная статистика по ребалансировкам для этого периода.
Можно сделать с 04.03.2011, но нет инф-ции по капитализации для тестирования фактора Size,
поэтому будут только Momentum, Low volatility и High dividends"""

***Создание свода о ребалансировках наших индексов + цены с дивидендами***

In [ ]:
# Добавить комиссии за ребалансировку!

In [192]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math

In [193]:
rebalancing = pd.read_excel('moex - assets.xlsx', sheet_name='динамика для бэктеста', header=1)
price = pd.read_excel('moex - assets.xlsx', sheet_name='цены', index_col=0).iloc[1:]
price = price.fillna(0)
dividends = pd.read_excel('moex - assets.xlsx', sheet_name='дивиденды').iloc[:, 1:]
Nshares = pd.read_excel('moex - assets.xlsx', sheet_name='количество акций', index_col=0)

In [194]:
# Функция берет столбец с ценами и возвращает столбец с изменениями цен без учета неторгуемых дней
def price_to_changes(price_path, log):
    price_path_f = price_path.copy()
    price_path_f = price_path_f[price_path_f != 0]
    if log == False:
        price_changes = price_path_f / price_path_f.shift(1) - 1
    else:
        price_changes = (price_path_f / price_path_f.shift(1)).apply(lambda x: math.log(x))
    price_changes = price_changes.dropna()
    price_changes = price_changes[price_changes != 0]
    
    return(price_changes)

In [195]:
# Функция берет дивидендную дату и, если дивиденд есть и ГЭП попадает в динамику цен, то находит примерную дату, когда был дивидендный ГЭП
def div_date_to_gap_date(div, div_date, price_changes):
    if not(np.isnan(div)):
        if (div_date >= price_changes.index[2]) and (div_date <= price_changes.index[-1]):
            div_date = price_changes[price_changes.index <= div_date].index[-1]
            possible_gap = price_changes[price_changes.index <= div_date][-3:]
            div_gap_date = possible_gap[possible_gap == possible_gap.min()].index[0]
        else:
            div_gap_date = 'За рамками динамики'
    else:
        div_gap_date = 'Нет дивиденда'
        
    return(div_gap_date)

In [196]:
# Функция добавляет в словарь параметры Моментума с указанным окном
# 'Недостаточная история торгов' - если не хватает истории котировок как минимум на половину окна моментума

def momentum_calc(selection, price, window):
    selection_f = selection.copy()
    window = 365 // 12 * window
    for date in selection_f.keys():
        param_calc = []

        for ticker in selection_f[date]['Актив']:
            dynamics = price[ticker]
            dynamics = dynamics[dynamics.index < date]
            dynamics = dynamics[dynamics.index > date - timedelta(window)]
            dynamics = dynamics[dynamics != 0]
            dif = (date - dynamics.index[0]).days

            if dif < window // 2:
                param_calc.append('Недостаточная история торгов')
            else:
                change = dynamics[-1] / dynamics[0] - 1
                param_calc.append(change)

        selection_f[date]['Momentum_' + str(int(12 / (365 // window))) + "M"] = param_calc
    
    return(selection_f)

In [197]:
# Функция добавляет в словарь параметры Volatility с указанным окном
# 'Недостаточная история торгов' - если не хватает истории котировок как минимум на половину окна Volatility

def volatility_calc(selection, price, window):
    selection_f = selection.copy()
    window = 365 // 12 * window
    for date in selection_f.keys():
        param_calc = []

        for ticker in selection_f[date]['Актив']:
            dynamics = price[ticker]
            dynamics = dynamics[dynamics.index < date]
            dynamics = dynamics[dynamics.index > date - timedelta(window)]
            dynamics = dynamics[dynamics != 0]
            dif = (date - dynamics.index[0]).days

            if dif < window // 2:
                param_calc.append('Недостаточная история торгов')
            else:
                change = price_to_changes(dynamics, log=True)
                vol = change.std() 
                param_calc.append(vol)

        selection_f[date]['Volatility_' + str(int(12 / (365 // window))) + "M"] = param_calc
    
    return(selection_f)

In [198]:
# Size - на момент балансировки: цена акции * кол-во акций
def size_calc(selection, price, Nshares):   
    selection_f = selection.copy()

    for date in selection_f.keys():
        param_calc = []
        for ticker in selection_f[date]['Актив']:
            number_shares = Nshares[Nshares.index <= date].iloc[-1][ticker]
            quote = price[price.index < date].iloc[-1][ticker]
            size = number_shares * quote
            param_calc.append(size)

        selection_f[date]['Size'] = param_calc
    
    return(selection_f)

In [199]:
# див. доходность считаем как сумма дивидендов за предыдущие 12 месяцев / цена акций на дату формирования реестра
def div_return_calc(selection, price, dividends):    
    selection_f = selection.copy()

    for date in selection_f.keys():
        date_year_ago = date - timedelta(365)
        param_calc = []
        for ticker in selection_f[date]['Актив']:
            try:
                divs_frame = dividends[(dividends['Date ' + ticker] < date) & (dividends['Date ' + ticker] > date_year_ago)]\
                                                 [['Date ' + ticker, 'Divs ' + ticker]].reset_index(drop=True)

                div_return = 0
                for d in range(len(divs_frame)):
                    div_date = divs_frame.iloc[d, 0]
                    div_value = divs_frame.iloc[d, 1]
                    stock_price = price[price.index == div_date][ticker].iloc[0]
                    if not np.isnan(stock_price) and stock_price != 0:
                        div_return += div_value / stock_price    
                param_calc.append(div_return)
            except:
                param_calc.append(0)
            
        selection_f[date]['Dividends'] = param_calc
        
    return(selection_f)

In [200]:
# Добавим в цену дивиденды. Все тикеры из таблицы с ценами должны быть в таблице с дивидендами, иначе выдаст ошибку.
tickers = price.columns.tolist()
for ticker in tickers:
    divs = dividends[['Date ' + ticker, 'Divs ' + ticker]]
    price_path = price[ticker].copy()
    price_changes = price_to_changes(price_path, log=False)
    for i in range(len(divs)):
        div_date = divs.iloc[i, 0]
        div = divs.iloc[i, 1]
        div_gap_date = div_date_to_gap_date(div, div_date, price_changes)
        if div_gap_date == 'Нет дивиденда':
            break
        if div_gap_date == 'За рамками динамики':
            continue
        else:
            price_path[price_path.index == div_gap_date] += div
    price[ticker] = price_path

In [201]:
# Частота ребалансировки (в месяцах)
freq = 3
freq_days = freq * 30
# Указываем окно для бэктеста (динамика)
# start_date должна быть на год больше, чем минимальная дата в котировках (для подсчета дивидендов за год)
# или freq месяцев больше, если не считаем фактор дивидендов
start_date = pd.to_datetime('18.12.2012')
end_date = pd.to_datetime('29.04.2022')

In [202]:
last_reb_date = start_date - timedelta(freq_days)
ac_reb_date = start_date
# min_days - столько дней должны торговаться половина компаний из индекса с момента последней ребалансировки
min_days = round(freq * 30 * 5 / 7 * 0.3)

In [203]:
# В словарь moex_rebs запишем по датам ребалансировок из нашего периода актуальные акции - Universe
universe1 = rebalancing.columns[(rebalancing.columns <= start_date).sum() - 1]
moex_rebs = {universe1 : rebalancing[universe1][[x in tickers for x in rebalancing[universe1]]].tolist()}
reb_dates = rebalancing.columns[(rebalancing.columns > start_date) & (rebalancing.columns < end_date)]
for reb_date in reb_dates:
    moex_rebs[reb_date] = rebalancing[reb_date][[x in tickers for x in rebalancing[reb_date]]].tolist()

In [204]:
# Ребалансировки индекса со списком активов, которые торговались в дату ребалансировки
real_rebs = {}
while ac_reb_date < end_date:

    moex_date = list(moex_rebs.keys())[sum([x <= ac_reb_date for x in list(moex_rebs.keys())]) - 1]
    ac_universe = moex_rebs[moex_date]
    not_trading = []

    Ntrade = 0
    Ntrade_last = 0
    for ticker in ac_universe:
        price_universe = price[ticker][price[ticker] != 0]
        price_universe = price_universe[price_universe.index <= ac_reb_date]
        price_universe = price_universe[price_universe.index >= last_reb_date]
        change_universe = (price_universe / price_universe.shift(1)).dropna()
        trade_days = (change_universe != 1).sum()
        if trade_days >= min_days:
            Ntrade += 1
        if change_universe[-1] != 1:
            Ntrade_last += 1
        else:
            not_trading.append(ticker)

    if Ntrade / len(ac_universe) >= 0.5:
        if Ntrade_last / len(ac_universe) >= 0.5:
            ac_universe = [x for x in ac_universe if x not in not_trading]
            real_rebs[ac_reb_date] = ac_universe
            last_reb_date = ac_reb_date
            ac_reb_date = ac_reb_date + timedelta(freq_days)
        else:
            ac_reb_date = ac_reb_date + timedelta(1)
    
    else:
        ac_reb_date = ac_reb_date + timedelta(freq_days)


***Расчет значений стратегий***

In [205]:
# В каждую дату ребалансировки положим датафрейм, куда будем складывать параметры стратегий через функции
selection = {}
for i in real_rebs.keys():
    selection[i] = pd.DataFrame(real_rebs[i], columns=['Актив'])

In [206]:
# Добавляем Моментум (можно несколько раз с разными окнами)
# windowM - окно в месяцах
windowM = 3
selection = momentum_calc(selection, price, windowM)

In [207]:
# Добавляем Volatility (можно несколько раз с разными окнами)
# windowV - окно в месяцах
windowV = 6
selection = volatility_calc(selection, price, windowV)

In [208]:
# Добавляем Size
selection = size_calc(selection, price, Nshares)

In [209]:
# Добавляем Dividends
selection = div_return_calc(selection, price, dividends)

***Отбор компаний по рассчитанным параметрам***

In [210]:
# Моментум и дивиденды прямо влияют на ожидаемую дох-ть, размер и волатильность - обратно, согласно предположениям
mom_direction = True
vol_direction = False
size_direction = False
div_direction = True

In [211]:
# Можно пробовать разные комбинации стратегий и кол-ва отбираемых из вселенной компаний.
# Для этого для каждой стратегии сначала вводим кол-во комбинаций, которые хотим протестировать,
# А затем для каждой комбинации - сколько компаний будем отбирать из вселенной
parameters = list(selection[list(selection.keys())[0]].columns[1:])
N_select = {}
for param in parameters:
    N_comb = int(input('Введите количество комбинаций для стратегии ' + param))
    list_comb = []
    for i in range(N_comb):
        comb = int(input('Введите количество отбираемых компаний для комбинации '\
                         + str(i+1) + ', для стратегии '+ param))
        list_comb.append(comb)
    N_select[param] = list_comb
    

Введите количество комбинаций для стратегии Momentum_3M2
Введите количество отбираемых компаний для комбинации 1, для стратегии Momentum_3M5
Введите количество отбираемых компаний для комбинации 2, для стратегии Momentum_3M10
Введите количество комбинаций для стратегии Volatility_6M2
Введите количество отбираемых компаний для комбинации 1, для стратегии Volatility_6M5
Введите количество отбираемых компаний для комбинации 2, для стратегии Volatility_6M10
Введите количество комбинаций для стратегии Size5
Введите количество отбираемых компаний для комбинации 1, для стратегии Size3
Введите количество отбираемых компаний для комбинации 2, для стратегии Size4
Введите количество отбираемых компаний для комбинации 3, для стратегии Size5
Введите количество отбираемых компаний для комбинации 4, для стратегии Size6
Введите количество отбираемых компаний для комбинации 5, для стратегии Size7
Введите количество комбинаций для стратегии Dividends1
Введите количество отбираемых компаний для комбинаци

In [212]:
final_select = {}
for date in selection.keys():
    final_select[date] = pd.DataFrame()
    select = selection[date]
    final_select[date]['Актив'] = select['Актив']
    
    for strat in list(select.columns)[1:]:
        select_strat = select[['Актив', strat]]
        select_strat = select_strat[select_strat[strat] != 'Недостаточная история торгов']
        if strat.__contains__('Momentum'):
            direction = mom_direction
        elif strat.__contains__('Volatility'):    
            direction = vol_direction
        elif strat.__contains__('Size'):     
            direction = size_direction
        elif strat.__contains__('Dividends'):    
            direction = div_direction
            
        # Для Size капитализацию по прив. акциям прибавляем к обычным, если они есть в выборке (обычно представлены обе)
        if strat == 'Size':
            for ticker in list(select_strat['Актив']):
                short = ticker[:-1]
                if short in list(select_strat['Актив']):
                    select_strat.loc[select_strat['Актив'] == short, 'Size'] +=\
                    select_strat.loc[select_strat['Актив'] == ticker, 'Size'].iloc[0]
                    select_strat = select_strat.loc[select_strat['Актив'] != ticker]

        select_strat = select_strat.sort_values(strat, ascending=not direction).reset_index(drop=True)

        for n_comp in N_select[strat]:
            top_assets = list(select_strat.iloc[:n_comp]['Актив'])
            top_assets = [x in top_assets for x in list(select['Актив'])]
            final_select[date][strat + '_' + str(n_comp) + 'Comp'] = top_assets

In [213]:
final_select

{Timestamp('2012-12-18 00:00:00'):     Актив  Momentum_3M_5Comp  Momentum_3M_10Comp  Volatility_6M_5Comp  \
 0    AFKS              False               False                False   
 1    AFLT              False               False                 True   
 2    AKRN              False               False                False   
 3    ALRS              False                True                False   
 4    BANE              False               False                False   
 5   BANEP              False               False                False   
 6    CHMF              False               False                False   
 7    UPRO              False               False                False   
 8    FEES              False               False                False   
 9    GAZP              False               False                False   
 10   GMKN              False                True                False   
 11   HYDR              False               False                False   
 12 

***Динамика портфелей***